In [1]:
import pandas as pd
import numpy as np


In [2]:
DATA_DIR = '../data/'
df = pd.read_csv(DATA_DIR+'Most-Recent-Cohorts-Institution.csv', low_memory=False)

In [3]:
df = df.replace('PrivacySuppressed', np.nan)


In [4]:
df.shape

(6543, 3232)

In [5]:
nan_percentage = df.isna().mean()

# Filter out columns with more than 30% NaN values
df_cleaned = df.loc[:, nan_percentage <= 0.3].copy()


In [6]:
df_cleaned.shape

(6543, 689)

In [7]:
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('float64')]))
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('int64')]))
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]))

363
15
311


In [8]:
# Find columns which cannot be coerced to object dtype. 
# These are actual categorical columns of potential interest.
err_cols = set()
for col in df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]:
    try:
        pd.to_numeric(df_cleaned[col], errors='raise')
    except:
        err_cols.add(col)


In [9]:
# Coerce whatever columns which can be coerced. These are numeric columns.
for column in df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]:
    if column not in err_cols:
        df_cleaned.loc[:, column] = pd.to_numeric(df_cleaned[column], errors='coerce')


In [81]:
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('float64')]))
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('int64')]))
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]))

615
15
59


In [31]:
# Borrower-Based Repayment Rate on Federal Loans: 
# BBRR[YR]_[LOAN]_[GROUP]_[STATUS]
# [YR]=1, 2, 3, or 4
# [STATUS]:
# DFTL - Default - Not paid for 360+ days
# DLNQ - Delinquent - Not paying for 31 to 360 days
# FBR - Forbearance - Temporary stop
# DFR - Deferment 
# NOPROG - Not making progress
# MAKEPROG - Making progress
# PAIDINFULL - Paid in full
# Discharged - Discharged - obligation to pay removed
# GROUP - UG, GR, UGCOMP. UGNOCOMP, GRCOMP, GRNOCOMP
# comp means completion 
# more info @ page 33-34

# questionable_cols = ['FEDSCHCD']
# ACCREDAGENCY <-- str to cat

# val_cat_cols = ['ACCREDAGENCY', 'ACCREDCODE']
# dates = ['SEPAR_DT_MDN', 'T4APPROVALDATE']
# ACCREDCODE is the code for ACCREDAGENCY, so one can be safely dropped
# IDS = ['UNITID']
# drop col = ['MDCOST_ALL', 'MDEARN_ALL'] # <== value is 16007 and 38461 respectively for all rows!


In [89]:
num_cols = set(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('float64')])
cat_cols = set(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]) | set(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('int64')])

cat_cols_to_drop = set([
    'FEDSCHCD', 'MDCO1ST_ALL', 'MDEARN_ALL',# no idea what these are
    'ACCREDAGENCY',                         # duplicate of ACCREDCODE
    'SEPAR_DT_MDN', 'T4APPROVALDATE',       # dates
    'INSTURL', 'NPCURL',                    # urls
    'UNITID',                               # ID column
    'ADDR', 'ZIP',                          # institute address
    'INSTNM'                                # institute name
])
cat_cols -= cat_cols_to_drop
cat_cols = [c for c in cat_cols if not c.startswith('BBRR')] # dropping BBRR for now
num_cols = [c for c in num_cols if not c.startswith('BBRR')]

In [95]:
# fill all num cols by median values
for column in df_cleaned[num_cols].columns:
    median_value = df_cleaned[column].median()
    df_cleaned[column].fillna(median_value, inplace=True)

# fill all cat cols by most frequent values
for column in df_cleaned[cat_cols].columns:
    mode_value = df_cleaned[column].mode()
    df_cleaned[column].fillna(mode_value, inplace=True)

In [98]:
df_cleaned = df_cleaned[num_cols + cat_cols]

In [99]:
df_cleaned

,COMP_ORIG_YR4_RT,GT_28K_P10,CIP25CERT1,LO_INC_YR3_N,CIP26CERT2,CCSIZSET,MN_EARN_WNE_P6,PCIP23,CIP09CERT2,UGDS_HISP,...,CONTROL_PEPS,ICLEVEL,PREDDEG,HIGHDEG,OPEFLAG,MAIN,MDCOST_ALL,NUMBRANCH,CITY,HCM2
0,0.222367,0.5670,0.0,1002.0,0.0,14.0,28400.0,0.0107,0.0,0.0114,...,Public,1,3,4,1,1,16007,1,Normal,0
1,0.454072,0.7260,0.0,1334.0,0.0,15.0,39400.0,0.0141,0.0,0.0669,...,Public,1,3,4,1,1,16007,1,Birmingham,0
2,0.132812,0.7000,0.0,122.0,0.0,6.0,35400.0,0.0000,0.0,0.0438,...,Private Nonprofit,1,3,4,1,1,16007,1,Montgomery,0
3,0.369397,0.7550,0.0,477.0,0.0,13.0,40300.0,0.0243,0.0,0.0610,...,Public,1,3,4,1,1,16007,1,Huntsville,0
4,0.246203,0.4940,0.0,805.0,0.0,14.0,24400.0,0.0098,0.0,0.0129,...,Public,1,3,4,1,1,16007,1,Montgomery,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538,0.573754,0.7910,0.0,4076.0,0.0,3.0,43600.0,0.0000,0.0,0.1081,...,Public,1,0,0,1,0,16007,23,York,0
6539,0.573754,0.7910,0.0,4076.0,0.0,3.0,43600.0,0.0000,0.0,0.1081,...,Public,1,0,0,1,0,16007,23,Malvern,0
6540,0.573754,0.7910,0.0,4076.0,0.0,3.0,43600.0,0.0000,0.0,0.1081,...,Public,1,0,0,1,0,16007,23,Middletown,0
6541,0.573754,0.7910,0.0,4076.0,0.0,3.0,43600.0,0.0000,0.0,0.1081,...,Public,1,0,0,1,0,16007,23,Media,0
